# Tools and Routing 🛠️ 

## Introduction 
This notebook demonstrates `the use of various Python tools` and libraries to perform tasks such as environment variable loading, HTTP requests, and data formatting. We will also define some custom functions and tools to interact with APIs and fetch data. The primary libraries used include os, openai, requests, and rich.


### Setup the Environment, OpenAI API Key  and Imports
First, we need to import the necessary libraries and set up the Environemnt and the OpenAI API key:

In [1]:
# Import necessary libraries
import os
import openai
from dotenv import load_dotenv, find_dotenv

# Load environment variables from .env file
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

# Print OpenAI API key (masked)
print(f"OPENAI_API_KEY: {os.getenv('OPENAI_API_KEY')[:5]}*****")

OPENAI_API_KEY: sk-ft*****


**Note** Ensure you have the required packages installed:
```py
%pip install pydantic==1.10.8
%pip install rich
```

In [2]:
# Import necessary modules from rich library that helps to improve the readability of nested dictionary outputs
from rich import print
from rich.pretty import Pretty

In [3]:
# Import necessary modules from Pydantic 
from typing import List
from pydantic import BaseModel, Field

# Import necessary modules from  Langchain to define custom tools.
from langchain.agents import tool

### Defining Custom Search Tool to search for Wheather online

In this example we will define a custom search tool to search for weather online.

In [4]:
# Define a custom search tool to search for weather online
@tool
def search(query: str) -> str:
    """Search for weather online"""
    return "42f"

#### Retrieving Search Tool Information

We will retrieve information about the custom search tool, including its name, description, and arguments.

In [5]:
# Get the name of the search tool
search.name

'search'

In [6]:
# Get the description of the search tool
search.description

'Search for weather online'

In [7]:
# Get the arguments of the search tool
search.args

{'query': {'title': 'Query', 'type': 'string'}}

#### Defining Input Schema for Search Tool

We will define an input schema for the search tool using the pydantic library.

In [8]:
#from pydantic import BaseModel, Field

# Define the input schema for the search tool
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")


#### Updating Search Tool with Input Schema

We will update the search tool to use the defined input schema.

In [9]:
@tool(args_schema=SearchInput)
def search(query: str) -> str:
    """Search for the weather online."""
    return "42f"

#### Retrieving Updated Search Tool Information

We will retrieve information about the updated search tool with the input schema.

In [10]:
# Get the arguments of the search tool with the input schema
search.args

{'query': {'description': 'Thing to search for',
  'title': 'Query',
  'type': 'string'}}

#### Running Search Tool

We will run the search tool with a sample query to see its output.

In [11]:
# Run the search tool with a sample query
search.run("sf")

'42f'

#### Defining Get Current Temperature Tool

Next, we will define a tool to fetch the current temperature for given coordinates using the requests library.

In [12]:
import requests
#from pydantic import BaseModel, Field
import datetime

# Define the input schema for the get_current_temperature tool
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")


# Define a tool to fetch current temperature for given coordinates
@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

#### Retrieving Get Current Temperature Tool Information  
We will retrieve information about the get_current_temperature tool, including its name, description, and arguments.

In [13]:
# Get the name of the get_current_temperature tool
get_current_temperature.name

'get_current_temperature'

In [14]:
# Get the description of the get_current_temperature tool
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [15]:
# Get the arguments of the get_current_temperature tool
get_current_temperature.args

# Pretty print the arguments of the get_current_temperature tool
print(Pretty(get_current_temperature.args))

{
    'latitude': {
        'description': 'Latitude of the location to fetch weather data for',
        'title': 'Latitude',
        'type': 'number'
    },
    'longitude': {
        'description': 'Longitude of the location to fetch weather data for',
        'title': 'Longitude',
        'type': 'number'
    }
}

#### Formatting Tool to OpenAI Function

We will format the get_current_temperature tool to an OpenAI function.

In [17]:
# Import necessary modules from  Langchain
from langchain.tools.render import format_tool_to_openai_function

In [18]:
# Format the get_current_temperature tool to OpenAI function
format_tool_to_openai_function(get_current_temperature)

# Print in a pretty format 
print(Pretty(format_tool_to_openai_function(get_current_temperature)))

C:\Users\Michela\AppData\Local\Temp\ipykernel_15124\692620006.py:2: LangChainDeprecationWarning: The function `_format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 1.0. Use :meth:`~langchain_core.utils.function_calling.convert_to_openai_function()` instead.
  format_tool_to_openai_function(get_current_temperature)


{
    'name': 'get_current_temperature',
    'description': 'Fetch current temperature for given coordinates.',
    'parameters': {
        'properties': {
            'latitude': {'description': 'Latitude of the location to fetch weather data for', 'type': 'number'},
            'longitude': {'description': 'Longitude of the location to fetch weather data for', 'type': 'number'}
        },
        'required': ['latitude', 'longitude'],
        'type': 'object'
    }
}

**Explanation output** This output represents the metadata for the `get_current_temperature tool`. It includes the tool's name, description, and parameters. The parameters are detailed with properties for `latitude` and `longitude`, both of type number, and marked as required.

In [20]:
get_current_temperature({"latitude": 13, "longitude": 14})

C:\Users\Michela\AppData\Local\Temp\ipykernel_15124\2143878976.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_current_temperature({"latitude": 13, "longitude": 14})
C:\Users\Michela\AppData\Local\Temp\ipykernel_15124\1967762575.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 30.4°C'

### Defining a Custom Wikipedia Search Tool

Here, let's  `define a custom tool to search Wikipedia and retrieve page summaries`. The tool uses the wikipedia library to perform the search and fetch the summaries of the top three results. If no suitable results are found, it returns a message indicating that no good Wikipedia search result was found. 

**Note** Ensure you have the required packages installed:
``` py 
%pip install wikipedia
```

In [21]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [22]:
search_wikipedia.name

'search_wikipedia'

In [23]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [24]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'properties': {'query': {'type': 'string'}},
  'required': ['query'],
  'type': 'object'}}

In [25]:
search_wikipedia({"query": "langchain"})

# Print in a pretty format 
print(Pretty(search_wikipedia({"query": "langchain"})))

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large 
language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely 
overlap with those of language models in general, including document analysis and summarization, chatbots, and code
analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval-Augmented Generation (RAG) is a technique 
that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions 
with a large language model (LLM) so that the model responds to user queries with reference to a specified set of 
documents, using this information to augment information drawn from its own vast, static training data. This allows
LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal 
company data or giving factual information only from an authoritative source.\n\n\n\nPage: Milvus (vector 
database)\nSummary: Milvus is a distributed vector database developed by Zilliz. It is available as both 
open-source software and a cloud service.\nMilvus is an open-source project under LF AI & Data Foundation 
distributed under the Apache License 2.0.\n\n"

**Explanation output** This output contains summaries of three different Wikipedia pages:

- `LangChain`: Describes LangChain as a software framework that integrates large language models (LLMs) into applications, with use-cases such as document analysis, chatbots, and code analysis.  
- `Milvus (vector database)`: Describes Milvus as a distributed vector database developed by Zilliz, available as open-source software and a cloud service, and part of the LF AI & Data Foundation.  
- `Retrieval-augmented generation`: Explains Retrieval-Augmented Generation (RAG) as a technique that enhances generative AI models with information retrieval capabilities, allowing them to provide responses based on specific documents.

In [27]:
# Import necessary modules from  Langchain
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [28]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

- Create an OpenAPISpec instance from a given text input

In [29]:
spec = OpenAPISpec.from_text(text)  # The 'text' variable should contain the OpenAPI specification as a string

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


- Convert the OpenAPI specification into OpenAI functions and callables using the provided spec

The function openapi_spec_to_openai_fn returns two outputs:    
  1. pet_openai_functions: A list or dictionary of OpenAI functions derived from the OpenAPI spec  
  2. pet_callables: A list or dictionary of callable functions derived from the OpenAPI spec  

In [30]:
# Convert the OpenAPI specification into OpenAI functions and callables using the provided spec
pet_openai_functions, pet_callables = openapi_spec_to_openai_fn(spec)

In [31]:
pet_openai_functions

# Print in a pretty format 
print(Pretty(pet_openai_functions))

[
    {
        'name': 'listPets',
        'description': 'List all pets',
        'parameters': {
            'type': 'object',
            'properties': {
                'params': {
                    'type': 'object',
                    'properties': {
                        'limit': {
                            'type': 'integer',
                            'maximum': 100.0,
                            'schema_format': 'int32',
                            'description': 'How many items to return at one time (max 100)'
                        }
                    },
                    'required': []
                }
            }
        }
    },
    {'name': 'createPets', 'description': 'Create a pet', 'parameters': {'type': 'object', 'properties': {}}},
    {
        'name': 'showPetById',
        'description': 'Info for a specific pet',
        'parameters': {
            'type': 'object',
            'properties': {
                'path_params': {
                    'type': 'object',
                    'properties': {'petId': {'type': 'string', 'description': 'The id of the pet to retrieve'}},
                    'required': ['petId']
                }
            }
        }
    }
]

**Explanation output** This output describes three API endpoints:  
- `listPets`: Lists all pets, with an optional parameter limit (integer, max 100) specifying how many items to return at once.  
- `createPets`: Creates a new pet without any specific parameters required.  
- `showPetById`: Retrieves information for a specific pet, requiring a petId parameter (string) to identify the pet.

In [33]:
# Import necessary modules from  Langchain
from langchain.chat_models import ChatOpenAI

In [34]:
model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)

C:\Users\Michela\AppData\Local\Temp\ipykernel_15124\1065844441.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(temperature=0).bind(functions=pet_openai_functions)


In [35]:
model.invoke("what are three pets names")

# Print in a pretty format 
print(Pretty(model.invoke("what are three pets names")))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"params":{"limit":3}}', 'name': 'listPets'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 17,
            'prompt_tokens': 123,
            'total_tokens': 140,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-e2cb3b7d-f505-4b4c-97e2-1c9a5117de9e-0'
)

In [37]:
# Invoke the model with a selected prompt 
model.invoke("tell me about pet with id 42")

# Print in a pretty format 
print(Pretty(model.invoke("tell me about pet with id 42")))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"path_params":{"petId":"42"}}', 'name': 'showPetById'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 20,
            'prompt_tokens': 126,
            'total_tokens': 146,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-919cff3c-7fdb-4b9f-8882-2f1af299a7b0-0'
)

### Routing

Given our tools above, let's format these as OpenAI functions and show the same behavior as it has been showed in `OpenAI Function Calling in LangChain` Notebobook, where it was showed an example of function calling deciding between two candidate functions.

- Create a list of formatted OpenAI functions from the `custom tools search_wikipedia` and `get_current_temperature`

In [39]:
# Create a list of formatted OpenAI functions from the custom tools search_wikipedia and get_current_temperature
functions = [
    format_tool_to_openai_function(f) for f in [
        search_wikipedia, get_current_temperature
    ]
]

# Instantiate a ChatOpenAI model with a specified temperature (0 in this case) and bind the formatted functions to it
model = ChatOpenAI(temperature=0).bind(functions=functions)

- Invoke the ChatOpenAI model with selected queries (e.g. "what is the weather in Amsterdam right now"). This will use the bound functions (e.g., search_wikipedia, get_current_temperature) to process the query

In [40]:
# Invoke the ChatOpenAI model with query1
model.invoke("what is the weather in Amsterdam right now")

# Print in a pretty format 
print(Pretty(model.invoke("what is the weather in Amsterdam right now")))

AIMessage(
    content='',
    additional_kwargs={
        'function_call': {
            'arguments': '{"latitude":52.3676,"longitude":4.9041}',
            'name': 'get_current_temperature'
        }
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 26,
            'prompt_tokens': 105,
            'total_tokens': 131,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-b3bd6c44-7f7e-40da-bc52-b659c85bada4-0'
)

In [42]:
# Invoke the ChatOpenAI model with query2
model.invoke("what is langchain")

# Print in a pretty format 
print(Pretty(model.invoke("what is langchain")))

AIMessage(
    content='',
    additional_kwargs={'function_call': {'arguments': '{"query":"Langchain"}', 'name': 'search_wikipedia'}},
    response_metadata={
        'token_usage': {
            'completion_tokens': 17,
            'prompt_tokens': 101,
            'total_tokens': 118,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-bf42d1f6-98ca-49a0-8c71-104100bb6c1d-0'
)

- Creation of a ChatPromptTemplate instance with predefined system and user messages

In [44]:
# Create a ChatPromptTemplate instance with predefined system and user messages
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

# Combine the prompt template with the ChatOpenAI model
chain = prompt | model

In [45]:
# Invoke the ChatOpenAI model with query3
chain.invoke({"input": "what is the weather in Amsterdam right now"})

# Print in a pretty format 
print(Pretty(chain.invoke({"input": "what is the weather in Amsterdam right now"})))

AIMessage(
    content='',
    additional_kwargs={
        'function_call': {
            'arguments': '{"latitude":52.3676,"longitude":4.9041}',
            'name': 'get_current_temperature'
        }
    },
    response_metadata={
        'token_usage': {
            'completion_tokens': 26,
            'prompt_tokens': 113,
            'total_tokens': 139,
            'completion_tokens_details': {
                'accepted_prediction_tokens': 0,
                'audio_tokens': 0,
                'reasoning_tokens': 0,
                'rejected_prediction_tokens': 0
            },
            'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}
        },
        'model_name': 'gpt-3.5-turbo',
        'system_fingerprint': None,
        'finish_reason': 'function_call',
        'logprobs': None
    },
    id='run-9afff3ee-63ec-4353-af9a-48b008340ce8-0'
)

- Creation of a chain that combines the prompt template, ChatOpenAI model, and OpenAIFunctionsAgentOutputParser

In [47]:
# Import necessary modules from  Langchain
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [48]:
# The chain processes the prompt through the model and parses the output using the specified parser
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [49]:
# Invoke the model with input1 
result = chain.invoke({"input": "what is the weather in Amsterdam right now"})

In [50]:
type(result)

langchain_core.agents.AgentActionMessageLog

In [51]:
result.tool

'get_current_temperature'

In [52]:
result.tool_input

{'latitude': 52.3676, 'longitude': 4.9041}

In [53]:
get_current_temperature(result.tool_input)

C:\Users\Michela\AppData\Local\Temp\ipykernel_15124\1967762575.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


'The current temperature is 4.0°C'

In [54]:
# Invoke the model with input2 
result = chain.invoke({"input": "hi!"})

In [55]:
type(result)

langchain_core.agents.AgentFinish

In [56]:
result.return_values

{'output': 'Well, hello there! How can I assist you today?'}

- Define a routing function to handle the result of an agent's action

In [57]:
# Import necessary modules from  Langchain
from langchain.schema.agent import AgentFinish

# Define a routing function to handle the result of an agent's action
def route(result):

    # Check if the result is an instance of AgentFinish
    if isinstance(result, AgentFinish):

        # If it is, return the final output value
        return result.return_values['output']
    else:

        # Otherwise, define a dictionary mapping tool names to their respective functions
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }

        # Run the appropriate tool function based on the result's tool name and input
        return tools[result.tool].run(result.tool_input)

-  Creation of a processing chain that:    
   1. Formats the prompt using the ChatPromptTemplate (prompt).  
   2. Passes the formatted prompt to the ChatOpenAI model (model).  
   3. Parses the output using OpenAIFunctionsAgentOutputParser.  
   4. Routes the parsed result to the appropriate function using the route function.  

In [58]:
# Create a processing chain that
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

- Testing 

In [59]:
# Invoke model using input 1 
result = chain.invoke({"input": "What is the weather in Amsterdam right now?"})

C:\Users\Michela\AppData\Local\Temp\ipykernel_15124\1967762575.py:34: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  current_utc_time = datetime.datetime.utcnow()


In [60]:
result

'The current temperature is 4.0°C'

In [61]:
# Invoke model using input 2
result = chain.invoke({"input": "What is langchain?"})

In [62]:
result
# Print the result function in a pretty format 
print(Pretty(result))

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large 
language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely 
overlap with those of language models in general, including document analysis and summarization, chatbots, and code
analysis.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval-Augmented Generation (RAG) is a technique 
that grants generative artificial intelligence models information retrieval capabilities. It modifies interactions 
with a large language model (LLM) so that the model responds to user queries with reference to a specified set of 
documents, using this information to augment information drawn from its own vast, static training data. This allows
LLMs to use domain-specific and/or updated information.  \nUse cases include providing chatbot access to internal 
company data or giving factual information only from an authoritative source.\n\n\n\nPage: Milvus (vector 
database)\nSummary: Milvus is a distributed vector database developed by Zilliz. It is available as both 
open-source software and a cloud service.\nMilvus is an open-source project under LF AI & Data Foundation 
distributed under the Apache License 2.0.\n\n"

In [64]:
# Invoke model using input 3 
chain.invoke({"input": "hi!"})

'Well, hello there! How can I assist you today?'

## Conclusion 
In this notebook, we have demonstrated how to set up and use various tools and libraries in Python for tasks such as loading environment variables, making HTTP requests, and formatting data. We defined `custom tools to interact with APIs and fetch data`, utilizing the langchain library for better integration and functionality. This structured approach to defining and using tools can simplify and streamline many common programming tasks.